In [1]:
import redis
import json
import time
import uuid
from gtts import gTTS
import os
from pathlib import Path
from IPython.display import Audio, display

In [2]:
r = redis.Redis()

In [3]:
audio_dir = Path("../public/audio")
audio_dir.mkdir(parents=True, exist_ok=True)

print("Worker started. Listening for tasks on Redis stream 'tts_stream'...")

Worker started. Listening for tasks on Redis stream 'tts_stream'...


In [ ]:
last_id = '0'
while True:
    try:
        entries = r.xread({'tts_stream':last_id}, block=0, count=1)
        for stream, messages in entries:
            for msg_id, message in messages:
                last_id = msg_id.decode()
                task_id = message[b'task_id'].decode()
                text = message[b'text'].decode()

                print(f"Received Task {task_id}: '{text}'")

                r.hset(f'tts_status:{task_id}', mapping={
                    'status': 'processing'
                })

                try: 
                    filename = f"{task_id}.mp3"
                    filepath = audio_dir / filename
                    # tts = gTTS(text)
                    tts = gTTS(text, lang='ur')
                    tts.save(str(filepath))
                    r.hset(f'tts_status:{task_id}', mapping={
                        'status': 'done',
                        'file': f'/audio/{filename}'
                    })
                    print(f"Task {task_id} done. File: {filepath}")
                    display(Audio(str(filepath)))
                except Exception as e:
                    r.hset(f'tts_status:{task_id}', mapping={
                        'status': 'error',
                        'message': str(e)
                    })
                    print(f"Error processing task {task_id}: {e}")
    except KeyboardInterrupt:
        print("worker stopped")
        break

Received Task eab38c53-c6fa-4ca7-bf90-c45f93e7d445: 'انڈیا کے وزیر اعظم نریندر مودی نے کہا ہے کہ ’تجارت اور دہشت گردی ایک ساتھ نہیں ہو سکتے اور نہ ہی پانی اور خون ایک ساتھ نہیں بہہ سکتا ہے۔ اگر پاکستان سے بات ہو گی تو دہشت گردی پر ہی ہو گی‘'
Task eab38c53-c6fa-4ca7-bf90-c45f93e7d445 done. File: ../public/audio/eab38c53-c6fa-4ca7-bf90-c45f93e7d445.mp3


Received Task 8587b939-ce88-4f9d-93b5-b31d80d9034a: 'کیا حال ہے تمہارا'
Task 8587b939-ce88-4f9d-93b5-b31d80d9034a done. File: ../public/audio/8587b939-ce88-4f9d-93b5-b31d80d9034a.mp3


Received Task 081e7559-d028-4911-92c6-cf328561fedf: 'لاہور‘ چینی کی قیمت میں ہوشربا اضافہ، شوگر ملز اور کریانہ ایسوسی ایشن آمنے سامنےاوپن مارکیٹ میں چینی کا ریٹ 164 سے بڑھ کر175 تک پہنچ گیا زائد اسٹاک نہ ہونے کے باوجود چینی کی برآمد کی اجازت دی، فائدہ صرف مخصوص عناصر نے اٹھایا‘کریانہ ایسوسیایشن'
Task 081e7559-d028-4911-92c6-cf328561fedf done. File: ../public/audio/081e7559-d028-4911-92c6-cf328561fedf.mp3


Received Task 157040b7-d903-47f1-ba39-6c028293e27f: 'کراچی(اردوپوائنٹ اخبار تازہ ترین۔14 مئی 2025) پاکستان کو عالمی مالیاتی فنڈ (آئی ایم ایف) سے موجودہ قرض پروگرام کی دوسری قسط موصول ہوگئی، سٹیٹ بینک آف پاکستان نے آئی ایم ایف سے ایک ارب 2 کروڑ 3 لاکھ ڈالر کی قسط موصول ہونے کی تصدیق کر دی، آئی ایم ایف کی جانب سے ای ایف ایف پروگرام کی دوسری قسط جاری کی گئی، سٹیٹ بینک کے مطابق آئی ایم ایف کی جانب سے دی گئی رقم 16 مئی کو ختم ہونے والے ہفتے میں ذخائر میں شامل کی جائے گی'
Task 157040b7-d903-47f1-ba39-6c028293e27f done. File: ../public/audio/157040b7-d903-47f1-ba39-6c028293e27f.mp3


Received Task 2434cf45-7b82-4e73-a38a-94974580e92b: 'نوازشریف کی شرافت، متانت،بردباری،معاملہ فہمی بزلہ سنجی،حاضر دماغی ،نوجوانوں کی حوصلہ افزائی،سیاسی بصیرت اور محفل کو خوشگوار رکھنے کیلئے فقرہ بازی نے پورے ہال کو اپنے سحر میں جکڑے رکھا.'
Task 2434cf45-7b82-4e73-a38a-94974580e92b done. File: ../public/audio/2434cf45-7b82-4e73-a38a-94974580e92b.mp3
